In [1]:
import torch
import numpy as np
import pandas as pd


In [2]:
df=pd.read_csv("C:\\Users\\shrey\\OneDrive\\Documents\\.PES\\PIL\\fin-dataset_2.csv")
df
df.drop(['node_code'],axis=1)

,Unnamed: 0,node_type,x(m),y(m),primary_channel,min_channel_allowed,max_channel_allowed,RSSI,SINR,throughput,average_airtime,average_interference,wlan_code_index
0,0,0,10.0000,10.0000,4,4,5,-58.226667,29.62,111.77,95.745,-107.399091,0
1,1,1,0.0713,10.8079,4,4,5,-65.370000,29.36,5.79,95.745,0.000000,0
2,2,1,1.9627,4.1427,4,4,5,-65.350000,29.48,6.11,95.745,0.000000,0
3,3,1,13.7849,16.7538,4,4,5,-61.410000,31.97,6.91,95.745,0.000000,0
4,4,1,6.7112,1.7487,4,4,5,-63.520000,31.26,9.99,95.745,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87904,87904,1,22.4444,46.9925,0,0,7,-68.170000,11.06,3.30,79.220,0.000000,9
87905,87905,1,32.0104,51.2173,0,0,7,-52.610000,32.17,43.24,79.220,0.000000,9
87906,87906,1,34.1608,48.5987,0,0,7,-59.770000,24.86,37.02,79.220,0.000000,9
87907,87907,1,31.8977,54.3313,0,0,7,-60.700000,23.68,36.86,79.220,0.000000,9


In [35]:
from sklearn.model_selection import train_test_split
y=df['throughput'].values #throughput 


x=df[[  'x(m)', 'y(m)',
       'primary_channel', 'min_channel_allowed', 'max_channel_allowed', 'RSSI',
       'SINR', 'average_airtime', 'average_interference',
       'wlan_code_index']].values
       

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=101)
print("PyTorch Version: ", torch.__version__)
       

PyTorch Version:  1.12.0+cu113


### Converting the dataframe into tensors for pytorch

In [36]:
#X_train.to_numpy()
X_train = torch.from_numpy(X_train).float()
X_train
y_train = torch.squeeze(torch.from_numpy(y_train).float())
X_test = torch.from_numpy(X_test).float()
y_test = torch.squeeze(torch.from_numpy(y_test).float())
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
y_test

torch.Size([70327, 10]) torch.Size([70327])
torch.Size([17582, 10]) torch.Size([17582])


tensor([ 2.5300,  2.8700,  0.0000,  ..., 14.7500, 11.2100,  4.4700])

#### Neural network

In [38]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):
  def __init__(self, n_features):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(n_features, 5)
    self.fc2 = nn.Linear(5, 3)
    self.fc3 = nn.Linear(3, 1) # result is a single number so only one output
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    #return torch.sigmoid(self.fc3(x))
    return x
# an instance
net = Net(X_train.shape[1])    

![the NN]("NN1.png")

#### Running it locally on gpu:

In [40]:

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001) # lr is taken to be a small value for trial purposes


In [41]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)
net = net.to(device)
criterion = criterion.to(device)

In [42]:
def calculate_accuracy(y_true, y_pred):
  predicted = y_pred#.ge(.5).view(-1)
  return (y_true == predicted).sum().float() / len(y_true)

### TRAINING:

In [44]:
def round_tensor(t, decimal_places=3):
  return round(t.item(), decimal_places)


for epoch in range(1000):
    y_pred = net(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    if epoch % 100 == 0:
      train_acc = calculate_accuracy(y_train, y_pred)
      y_test_pred = net(X_test)
      y_test_pred = torch.squeeze(y_test_pred)
      test_loss = criterion(y_test_pred, y_test)
      test_acc = calculate_accuracy(y_test, y_test_pred)
      print(
f'''epoch {epoch}
Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)}
Test  set - loss: {round_tensor(test_loss)}, accuracy: {round_tensor(test_acc)}
''')

optimizer.zero_grad()
train_loss.backward()
optimizer.step()

RuntimeError: The size of tensor a (3) must match the size of tensor b (70327) at non-singleton dimension 1